In [ ]:
import pandas as pd
import scipy.stats as st
from datetime import datetime
from datetime import timedelta
import math
import numpy as np
from PIL import Image
from scipy import stats

In [ ]:
patient_route = pd.read_csv('covid19/PatientRoute.csv')
patient_info = pd.read_csv('covid19/PatientInfo.csv')
patient_info = patient_info[["patient_id","global_num","sex","age","infection_case"]]

In [ ]:
merge_route = pd.merge(patient_info, patient_route)
merge_route = merge_route[merge_route["province"]=="Seoul"]
merge_route = merge_route[pd.notnull(merge_route['sex'])]
merge_route = merge_route[pd.notnull(merge_route['age'])]

In [ ]:
causes = merge_route['infection_case']
causes = causes.drop_duplicates(keep='last')
causes = causes.dropna()
causes = causes.tolist()

causes_stay = ["contact with patient", "overseas inflow", "etc"]
for i, cause in enumerate(causes):
    if cause in causes_stay:
        continue
    causes[i] = "community infection"
    
causes = list(dict.fromkeys(causes))

In [ ]:
class GridMap:
    Re = 6371.00877  ##  지도반경
    grid = 1.28  ##  격자간격 (km)
    slat1 = 30.0  ##  표준위도 1
    slat2 = 60.0  ##  표준위도 2
    olon = 127.0175205  ##  기준점 경도
    olat = 37.523124  ##  기준점 위도
    xo = 18.4 / grid #  기준점 X좌표
    yo = 21.6 / grid  ##  기준점 Y좌표
    row = 32
    column = 32
    seoul_start_lat = 37.698098
    seoul_start_lon = 126.799791
    seoul_end_lat = 37.34815
    seoul_end_lon = 127.23525

    def __init__(self):
        self.PI = math.asin(1.0) * 2.0
        self.DEGRAD = self.PI / 180.0
        self.RADDEG = 180.0 / self.PI

        self.re = self.Re / self.grid
        self.slat1 = self.slat1 * self.DEGRAD
        self.slat2 = self.slat2 * self.DEGRAD
        self.olon = self.olon * self.DEGRAD
        self.olat = self.olat * self.DEGRAD

        self.sn = math.tan(self.PI * 0.25 + self.slat2 * 0.5) / \
                  math.tan(self.PI * 0.25 + self.slat1 * 0.5)
        self.sn = math.log(math.cos(self.slat1) /
                           math.cos(self.slat2)) / math.log(self.sn)
        self.sf = math.tan(self.PI * 0.25 + self.slat1 * 0.5)
        self.sf = math.pow(self.sf, self.sn) * math.cos(self.slat1) / self.sn
        self.ro = math.tan(self.PI * 0.25 + self.olat * 0.5)
        self.ro = self.re * self.sf / math.pow(self.ro, self.sn)

    def toGrid(self, lat, lon):
        lat = float(lat)
        lon = float(lon)
        ra = math.tan(self.PI * 0.25 + lat * self.DEGRAD * 0.5)
        ra = self.re * self.sf / pow(ra, self.sn)
        theta = lon * self.DEGRAD - self.olon
        if theta > self.PI:
            theta -= 2.0 * self.PI
        if theta < self.PI * -1:
            theta += 2.0 * self.PI
        theta *= self.sn
        x = (ra * math.sin(theta)) + self.xo
        y = (self.ro - ra * math.cos(theta)) + self.yo
        x = int(x + 1.5)
        # y = int(y + 1.5)
        y = (int(y + 1.5) - 31) * -1
        return x, y

    def toCoordinate(self, x, y):
        x = x - 1
        y = y - 1
        xn = x - self.xo
        yn = self.ro - y + self.yo
        ra = math.sqrt(xn * xn + yn * yn)
        if self.sn < 0.0:
            ra = -ra
        alat = math.pow((self.re * self.sf / ra), (1.0 / self.sn))
        alat = 2.0 * math.atan(alat) - self.PI * 0.5
        if math.fabs(xn) <= 0.0:
            theta = 0.0
        else:
            if math.fabs(yn) <= 0.0:
                theta = self.PI * 0.5
                if xn < 0.0:
                    theta = -theta
            else:
                theta = math.atan2(xn, yn)
        alon = theta / self.sn + self.olon
        lat = alat * self.RADDEG
        lon = alon * self.RADDEG
        return lat, lon

    def isSeoul(self, lat, lon):
        if self.seoul_end_lat > lat or lat > self.seoul_start_lat:
            return False
        if self.seoul_end_lon < lon or lon < self.seoul_start_lon:
            return False
        return True

    def generateMap(self, raw_data):
        gridx, gridy = [], []
        for idx, data in raw_data.iterrows():
            try:
                float(data.latitude)
            except ValueError:
                continue
            if self.isSeoul(data.latitude, data.longitude):
                x, y = self.toGrid(data.latitude, data.longitude)
                gridx.append(x), gridy.append(y)

        df = pd.DataFrame({'gridx': gridx, 'gridy': gridy})

        grid_array = np.zeros((self.row, self.column))
        for idx, data in df.iterrows():
            try:
                grid_array[data.gridy, data.gridx] += 1
            except:
                pass

        return grid_array

In [ ]:
class Feature:
    def __init__(self, names, counts, move_types, total_length):
        self.names = names
        self.counts = counts
        self.move_types = move_types
        self.clear(total_length)
        
    def clear(self, total_length):
        self.features = []
        all_count = sum(count for count in self.counts)
        self.tensor = np.zeros((total_length, 32, 32, all_count), dtype=np.float)
        for i in range(all_count):
            arr = np.zeros((32, 32))
            self.features.append(arr)
            
    def age_category(self, ages):
        categories = []
        for age in ages:
            age = int(age[:-1])
            if age == 0: category = 0
            elif age == 100: category = 10
            else: category = age // 10
            categories.append(category)
        return categories
            
    def sex_category(self, sexes):
        categories = []
        for sex in sexes:
            if sex == 'male': categories.append(0)
            else: categories.append(1)
        return categories
        
    def infection_case_category(self, infection_cases):
        categories = []
        for infection_case in infection_cases:
            if infection_case not in causes:
                infection_case = "community infection"
            category = causes.index(infection_case)
            categories.append(category)
        return categories
    
    def type_category(self, types):
        categories = []
        for purpose in types:
            category = move_types.index(purpose)
            categories.append(category)
        return categories
    
    def day_category(self, days):
        categories = []
        for day in days:
            day = datetime.strptime(day, "%Y-%m-%d")
            category = day.weekday()
            categories.append(category)
        return categories
    
    def put_all_categories(self, order, ages, sexes, infection_cases, types, days, lats, lons):
        converter = GridMap()
        length = len(ages)
        
        ages_categories = self.age_category(ages)
        sexes_categories = self.sex_category(sexes)
        infection_case_categories = self.infection_case_category(infection_cases)
        type_categories = self.type_category(types)
        days_categories = self.day_category(days)
        
        max_in_day = 0        
        
        # age_categories = 3이라고 했을 때
        # 전체 feature 배열에서 index가 3인 배열에 넣으면 됨
        for i in range(length):
            x, y = converter.toGrid(lats[i], lons[i])
            if x < 0 or y < 0 or x >= 32 or y >= 32: continue
            
            index = 0
            # age
            age = ages_categories[i]
            self.tensor[order][x][y][age] += 1
            if self.tensor[order][x][y][age] > max_in_day: max_in_day = self.tensor[order][x][y][age]
            index += self.counts[0]
            # sex
            if sexes_categories[i] != -1:
                sex = index + sexes_categories[i]
                self.tensor[order][x][y][sex] += 1
                if self.tensor[order][x][y][sex] > max_in_day: max_in_day = self.tensor[order][x][y][sex]
                index += self.counts[1]
            # infection_case
            if infection_case_categories[i] != -1:
                infection_case = index + infection_case_categories[i]
                self.tensor[order][x][y][infection_case] += 1
                if self.tensor[order][x][y][infection_case] > max_in_day: max_in_day = self.tensor[order][x][y][infection_case]
                index += self.counts[2]
            # type
            if type_categories[i] != -1:
                visit = index + type_categories[i]
                self.tensor[order][x][y][visit] += 1
                if self.tensor[order][x][y][visit] > max_in_day: max_in_day = self.tensor[order][x][y][visit]
                index += self.counts[3]
            # day
            day = index + days_categories[i]
            self.tensor[order][x][y][day] += 1
            if self.tensor[order][x][y][day] > max_in_day: max_in_day = self.tensor[order][x][y][day]
            
        return max_in_day

In [ ]:
def generate_images(feature_class, input_route, input_dates, total_length):
    feature_class.clear(total_length)
    
    max_in_tensor = 0
    for i in range(len(input_dates)):     
        routes = input_route.loc[input_route['date'] == input_dates[i]]

        # features in routes
        ages = routes['age'].tolist()
        sexes = routes['sex'].tolist()
        infection_cases = routes['infection_case'].tolist()
        types = routes['type'].tolist()
        dates = routes['date'].tolist()
        lats = routes['latitude'].tolist()
        lons = routes['longitude'].tolist()
        
        # features
        max_in_day = feature_class.put_all_categories(i, ages, sexes, infection_cases, types, dates, lats, lons)
        if max_in_day > max_in_tensor:
            max_in_tensor = max_in_day
        
    return feature_class.tensor, max_in_tensor

In [ ]:
route_dates = merge_route['date']
route_dates = route_dates.drop_duplicates(keep='last')
route_dates = route_dates.tolist()

In [ ]:
names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [11, 2, 4, 21, 7]
move_types = ['karaoke', 'gas_station', 'gym', 'bakery', 'pc_cafe',
              'beauty_salon', 'school', 'church', 'bank', 'cafe',
              'bar', 'post_office', 'real_estate_agency', 'lodging',
              'public_transportation', 'restaurant', 'etc', 'store',
              'hospital', 'pharmacy', 'airport']

In [ ]:
from keras.models import Sequential, load_model
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import keras.backend.tensorflow_backend as K
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pylab as plt

In [ ]:
# We create a layer which take as input movies of shape
# (n_frames, width, height, channels) and returns a movie
# of identical shape.
def get_model(channel):
    with K.tf_ops.device('/device:GPU:0'):
        seq = Sequential()
        # first
        seq.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
                           input_shape=(3, 32, 32, channel), data_format='channels_last',
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        # second
        seq.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        # thrid
        seq.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        # forth
        seq.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        # last
        seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
                       activation='sigmoid',
                       padding='same', data_format='channels_last'))

#         seq.compile(loss='binary_crossentropy', optimizer='adadelta')
        # compile이 뭐하는 애인지 알아둘 것
        seq.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    return seq

In [ ]:
feature = Feature(names, counts, move_types, len(route_dates))
image, max_in_image1 = generate_images(feature, merge_route, route_dates, len(route_dates))

In [ ]:
n = image.shape[0]
channel = image.shape[3]
rs = 32
n_step = 3
n_test = 1

train = image
    
X_train = []
y_train = []
    
for i in range(n - n_step):
    X_train.append(train[i:i+n_step:, :, :])
    y_train.append(train[i+n_step:i+n_step+n_test, :, :])

X_train, y_train = np.array(X_train), np.array(y_train)
print(X_train.shape, y_train.shape)

In [ ]:
seq = get_model(channel)
seq.fit(X_train, y_train, epochs = 100, batch_size = 32)